In [6]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

from glob import glob
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def collect_image_data(directory):
    images = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지를 RGB 형식으로 변환

            # 이미지 파일 이름에 따라 라벨링 수행
            if "open" in filename:
                label = "open"
            elif "close" in filename:
                label = "close"
            else:
                label = "unknown"

            images.append(image)
            labels.append(label)

    return images, labels

In [ ]:
def preprocess_image_data(images, labels, target_size=(64, 64)):
    # 이미지 크기 조정
    resized_images = [cv2.resize(image, target_size) for image in images]
    # 중복값 제거 (중복된 이미지 제거)
    unique_images, unique_labels = remove_duplicates(resized_images, labels)
    # 결측치 제거 (누락된 값이 있는 경우 해당 이미지 및 라벨 제거)
    cleaned_images, cleaned_labels = remove_missing_values(unique_images, unique_labels)
    # 이상치 제거 (이상치가 포함된 이미지 및 라벨 제거)
    outlier_free_images, outlier_free_labels = remove_outliers(cleaned_images, cleaned_labels)
    # 정규화 (모든 이미지를 0과 1 사이의 값으로 정규화)
    preprocessed_images = normalize_images(outlier_free_images)
    # 라벨을 숫자로 변환 (예: open -> 0, close -> 1)
    preprocessed_labels = convert_labels_to_numeric(outlier_free_labels)
    
    return preprocessed_images, preprocessed_labels

def remove_duplicates(images, labels):
    # 중복된 이미지를 제거하는 함수
    unique_images = []
    unique_labels = []
    seen = set()
    for i, image in enumerate(images):
        image_data = image.tostring()
        if image_data not in seen:
            seen.add(image_data)
            unique_images.append(image)
            unique_labels.append(labels[i])
    return unique_images, unique_labels

def remove_missing_values(images, labels):
    # 결측치(누락된 값)가 있는 이미지를 제거하는 함수
    cleaned_images = []
    cleaned_labels = []
    for i, image in enumerate(images):
        if image is not None and labels[i] is not None:
            cleaned_images.append(image)
            cleaned_labels.append(labels[i])
    return cleaned_images, cleaned_labels

def remove_outliers(images, labels):
    # 이상치 검출 및 제거 (여기서는 간단히 이미지 개수가 너무 적거나 많은 경우를 이상치로 처리)
    if len(images) < 10 or len(images) > 10000:
        return [], []
    else:
        return images, labels

def normalize_images(images):
    # 이미지를 0과 1 사이의 값으로 정규화하는 함수
    scaler = MinMaxScaler()
    flattened_images = [image.flatten() for image in images]
    normalized_images = scaler.fit_transform(flattened_images)
    return normalized_images.reshape(len(images), *images[0].shape)

def convert_labels_to_numeric(labels):
    # 라벨을 숫자로 변환하는 함수
    label_mapping = {"open": 0, "close": 1}
    numeric_labels = [label_mapping[label] for label in labels]
    return np.array(numeric_labels)

In [ ]:
# 모델 학습 함수
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# 결과 출력 함수
def predict_and_display(image):
    # 이미지 전처리
    preprocessed_image = preprocess_data(image)
    # 모델 사용하여 예측
    prediction = model.predict(preprocessed_image)
    if prediction > 0.5:
        print("눈을 뜨고 있습니다!")
    else:
        print("눈을 감고 있습니다!")

In [ ]:
# 데이터 수집
data_dir = 'data'  # 데이터 디렉토리 경로
images, labels = collect_data(data_dir)

# 데이터 전처리 및 모델 학습을 위한 데이터 준비
X_train = preprocess_data(images)
y_train = labels

# 모델 생성
input_shape = X_train[0].shape
model = create_model(input_shape)

# 모델 학습
train_model(model, X_train, y_train)

# 새로운 이미지에 대한 예측 및 결과 출력
predict_and_display(new_image)